# Telecom Conversation Topic Modeling Pipeline

## Overview
This notebook implements an end-to-end topic modeling pipeline for telecom customer service conversations using **BERTopic** with GPU acceleration. The goal is to automatically categorize customer conversations into meaningful topics to understand the dataset.

## Key Components

### **Technology Stack**
- **BERTopic**: Advanced topic modeling with transformer embeddings
- **RAPIDS cuDF/cuML**: GPU-accelerated data processing and machine learning
- **Sentence Transformers**: High-quality text embeddings (`all-MiniLM-L6-v2`)
- **UMAP**: Dimensionality reduction for clustering
- **HDBSCAN**: Density-based clustering for topic discovery

### **Pipeline Steps**
1. **Data Loading**: Import telecom conversation dataset using GPU-accelerated cuDF
2. **Text Embedding**: Generate semantic embeddings using pre-trained sentence transformers
3. **Vocabulary Filtering**: Extract meaningful words (frequency ≥ 15) to improve topic quality
4. **Model Training**: Configure and train BERTopic with optimized parameters
5. **Topic Inference**: Apply trained model to categorize all conversations
6. **Export Results**: Save topics and labeled conversations for downstream analysis

### **Model Configuration**
- **UMAP**: 5 components, 50 neighbors, cosine similarity
- **HDBSCAN**: Min cluster size 20, density-based clustering
- **Vocabulary**: Filtered for words appearing ≥15 times
- **Embeddings**: 384-dimensional vectors from sentence transformers

### **Processing Approaches**
The notebook demonstrates three different approaches for large-scale processing:
1. **Batch Processing**: Memory-efficient chunked processing (8 batches)
2. **Progress Tracking**: Single-pass processing with tqdm progress bars
3. **Simplified Apply**: Direct pandas apply function

### **Output**
- **Topics CSV**: Detailed topic information with representative words
- **Labeled Dataset**: Original conversations with assigned topic labels
- **Saved Model**: Serialized BERTopic model for future inference



In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telecom/aggregated_conversations.csv


In [ ]:
%%capture
# Install required packages for topic modeling with GPU acceleration

!pip install git+https://github.com/MaartenGr/BERTopic.git@master


!pip install cudf-cu12 dask-cudf-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cuml-cu12 --extra-index-url=https://pypi.nvidia.com  # GPU-accelerated machine learning
!pip install cugraph-cu12 --extra-index-url=https://pypi.nvidia.com  # GPU-accelerated graph analytics
!pip install cupy-cuda12x -f https://pip.cupy.dev/aarch64  # GPU array library


!pip install safetensors  # Secure tensor serialization
!pip install datasets  # Hugging Face datasets library
!pip install datashader  # Large data visualization
!pip install adjustText  # Automatic text placement for plots

In [ ]:
# Import libraries for topic modeling and text processing

from cuml.manifold import UMAP
from cuml.cluster import HDBSCAN

# Topic modeling and sentence embeddings
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# Utilities for text processing and progress tracking
import collections
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Configure locale and import GPU-accelerated dataframe library
import locale
import cudf

# Set UTF-8 encoding to handle multilingual text properly
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# Load the telecom conversation dataset using GPU-accelerated cuDF
train_gf = cudf.read_csv('/kaggle/input/telecom/aggregated_conversations.csv')

# Display basic information about the dataset
print('train shape is', train_gf.shape )
train_gf.head()

train shape is (228414, 2)


,conversation_id,text
0,98a6dbc997614ceb8e0734dda38b2524,"agent: Thank, thank you for calling Union Mobi..."
1,0e79164e7b8b4633bc1424637eacab32,"agent: Good afternoon, thank you for calling U..."
2,9db3dbf522c3487691ce42dd23658e0a,"agent: Good afternoon, thank you for calling U..."
3,4562ec45ff3e433b8dac72d215c9124c,"client: Hi, I'm having some issues with my pho..."
4,6bb5af1b731e485794dbaf4975fe78e9,"agent: Good afternoon, thank you for calling U..."


In [ ]:
from sentence_transformers import SentenceTransformer

# Initialize a pre-trained sentence transformer model for generating embeddings
# all-MiniLM-L6-v2 is a lightweight model that provides good quality embeddings
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings for all text conversations
embeddings = model.encode(train_gf['text'].to_arrow().to_pylist(), show_progress_bar=True)

In [ ]:
import numpy as np

# Save the generated embeddings to disk for reuse
# This avoids regenerating embeddings if the notebook is restarted
with open('/kaggle/working/embeddings.npy', 'wb') as f:
    np.save(f, embeddings)

In [ ]:
# Load pre-computed embeddings from disk (useful for notebook restarts)
embeddings = np.load('/kaggle/working/embeddings.npy')

In [ ]:
import collections
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

# Extract vocabulary from the corpus to be used in BERTopic
# This helps improve topic coherence by focusing on meaningful words
vocab = collections.Counter()
tokenizer = CountVectorizer().build_tokenizer()

# Tokenize all documents and count word frequencies
for doc in tqdm(train_gf['text'].to_arrow().to_pylist()):
  vocab.update(tokenizer(doc))

# Filter vocabulary to include only words that appear at least 15 times
# This removes rare words that might not contribute to meaningful topics
vocab = [word for word, frequency in vocab.items() if frequency >= 15]; len(vocab)

In [ ]:
from cuml.manifold import UMAP
from cuml.cluster import HDBSCAN
from bertopic import BERTopic

# Configure BERTopic sub-models for optimal performance

# Embedding model: Convert text to dense vector representations
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# UMAP: Reduce dimensionality while preserving local structure
# n_components=5: Reduce to 5 dimensions for clustering
# n_neighbors=50: Balance between local and global structure
# metric="cosine": Use cosine similarity for text data
umap_model = UMAP(n_components=5, n_neighbors=50, random_state=42, metric="cosine", verbose=True)

# HDBSCAN: Density-based clustering that can find clusters of varying sizes
# min_cluster_size=20: Minimum number of documents needed to form a cluster
# min_samples=20: Minimum samples in neighborhood for core points
hdbscan_model = HDBSCAN(min_samples=20, gen_min_span_tree=True, prediction_data=True, min_cluster_size=20, verbose=True)

# Vectorizer: Convert text to word count vectors using filtered vocabulary
vectorizer_model = CountVectorizer(vocabulary=vocab, stop_words="english")

# Initialize and train BERTopic model with all components
topic_model= BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        verbose=True
).fit(train_gf['text'].to_arrow().to_pylist(), embeddings=embeddings)

In [ ]:
# Display topic information including topic IDs, counts, and representative words
topic_model.get_topic_info()

In [ ]:
# Save the trained BERTopic model for later use
topic_model.save(
    path='/kaggle/working/model_dir',  # Directory to save the model
    serialization="safetensors",  # Use safetensors for secure serialization
    save_ctfidf=True,  # Save class-based TF-IDF representations
    save_embedding_model="sentence-transformers/all-MiniLM-L6-v2"  # Save embedding model reference
)

In [ ]:
# Convert topic information to a pandas DataFrame for easier manipulation and export
df_topic = pd.DataFrame(topic_model.get_topic_info())

In [ ]:
# Export topic information to CSV for external analysis and reporting
df_topic.to_csv('/kaggle/working/topics.csv', index=False)

In [ ]:
# Load the saved BERTopic model for inference on new text data
topic_model_inference = BERTopic.load("/kaggle/working/model_dir")

In [7]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')


In [ ]:
# Test the trained model on a sample conversation about phone service issues
sample_conversation = "agent: Hello, thank you for calling Union Mobile's 24/7 Phone Support. My name is Kristin, how can I assist you today? client: Hi Kristin, I'm having some issues with my phone service. I've been experiencing dropped calls, poor reception, and data connectivity problems. I've tried restarting my phone and checking my settings, but the issues persist. I'm getting really frustrated because I was promised that this would be resolved by now. agent: Sorry to hear that, Aurelia. I understand how frustrating it must be to have ongoing issues with your phone service. Can you tell me a little bit more about the problems you're experiencing? For example, are there having trouble making or receiving calls, or are you having difficulty accessing data services like internet browsing or social media? client: Yeah, it's mostly with making and receiving calls. Sometimes the calls just drop suddenly, and other times the person on the other end can't hear me properly. And when I try to access data services, it's really slow or doesn't work at all. It's like my phone is constantly searching for a signal. agent: I see. It sounds like you might be experiencing some network issues in your area. Let me check on that for you. Can you please tell me your location so I can look into this further? client: Sure, I'm in downtown Los Angeles. agent: Alright, I've checked on the network status in your area, and it looks like there have been some technical difficulties that may be causing the issues you're experiencing. Our engineering team is working on resolving the problem as we speak, but in the meantime, I can offer you a few solutions that might help improve your service. Would you like me to go over those with you? client: Yes, please do I just want my phone to work properly. agent: Of course, I completely understand. One thing you can try is turning your phone off and then back on again. This will allow help reset the connection and improve service. Additionally, you can try manually selecting a different network mode, such as switching from 4G to 3G or vice versa. This can sometimes help your phone latch onto a stronger signal. client: Okay, I'll try that. But what about data services? How do I get that to work properly? agent: For data services, you can try turning your phone's Wi-Fi off and then back on. This can sometimes help your phone reconnect to the nearest cell tower and improve data speeds. You can also try closing any apps that might be running in the background and using up data. This can help free up resources and improve overall performance. client: Alright, I'll give that a shot. Thank you, Kristin. agent: You're welcome, Aurelia. I apologize again for the inconvenience you've experienced, and I appreciate your patience. If you continue to have issues after trying these solutions, please don't hesitate to reach out to us again. We'll be happy to further you further. Is there anything else I can help you with today? client: No, that's all for now. Thanks again, Kristin. agent: You're welcome. Aurelia. Have a great day."

# Get topic prediction and probability for the sample conversation
topic, prob = topic_model_inference.transform(sample_conversation)

# Display the predicted topic label for this conversation
topic_model_inference.topic_labels_[topic[0]]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-28 05:49:12,364 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


'3_area_slow_speeds_network'

In [ ]:
# Display the loaded topic model object for inspection
topic_model_inference

In [ ]:
import cudf
import pandas as pd
from tqdm import tqdm  # tqdm is a library that provides a progress meter

# Batch processing approach to apply topic modeling to large dataset
texts_pd_series = train_gf['text'].to_pandas()

# Calculate batch parameters for processing in chunks to manage memory
total_records = len(train_gf)
records_per_chunk = total_records // 8  # Process data in 8 batches

def get_topic_label(text):
    """Extract topic label for a given text using the trained model"""
    topic, prob = topic_model_inference.transform(text)
    return topic_model_inference.topic_labels_[topic[0]]

def apply_and_print_progress(chunk_series):
    """Process a chunk of texts and track progress"""
    count = 0
    topic_labels = []
    for text in chunk_series:
        count += 1
        if count % 100 == 0:  # Print progress every 100 records
            print(f"Processing record {count}")
        topic_labels.append(get_topic_label(text))
    return topic_labels

# Initialize topic_label column in the main DataFrame
train_gf['topic_label'] = ''

# Process data in batches to avoid memory issues
for i in range(8):
    start_idx = i * records_per_chunk
    end_idx = start_idx + records_per_chunk if i < 7 else total_records

    # Extract the current chunk for processing
    chunk_series = texts_pd_series[start_idx:end_idx]

    # Display batch information
    print(f'Processing batch {i+1}/{8} with start_idx: {start_idx} and end_idx: {end_idx - 1}') # -1 because end_idx is exclusive

    # Process the chunk and track progress
    topic_labels_chunk = apply_and_print_progress(chunk_series)

    # Assign processed labels back to the main DataFrame
    train_gf.loc[start_idx:end_idx, 'topic_label'] = cudf.Series(topic_labels_chunk)

    # Save each processed chunk to CSV for backup
    filename = f'processed_chunk_{i + 1}.csv'
    train_gf.iloc[start_idx:end_idx].to_csv("/kaggle/working/filename", index=False)
    
    print(f'Batch {i+1}/{8} processed and uploaded.')

Processing batch 1/8 with start_idx: 0 and end_idx: 28550


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-28 05:54:01,811 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-28 05:54:01,839 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


...

2025-01-28 05:55:58,959 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-28 05:55:58,982 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-28 05:55:59,006 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


In [ ]:
import cudf
import pandas as pd
from tqdm import tqdm 

# Convert cuDF text column to pandas for processing
texts_pd_series = train_gf['text'].to_pandas()

total_records = len(train_gf)
records_per_chunk = total_records // 8

def get_topic_label(text):
    """Extract topic label for a given text using the trained model"""
    topic, prob = topic_model_inference.transform(text)
    return topic_model_inference.topic_labels_[topic[0]]

train_gf['topic_label'] = ''

def process_chunk(chunk_series):
    """Process a chunk of texts and return topic labels"""
    topic_labels = [get_topic_label(text) for text in chunk_series]
    return topic_labels


count = 0

def apply_and_print_progress(text):
    """Wrapper function to track progress during processing"""
    global count
    count += 1
    if count % 100 == 0:  # Print progress every 100 records
        print(f"Processing record {count}")
    return get_topic_label(text)

# Convert texts to list for efficient processing with tqdm
texts_list = texts_pd_series.tolist()

# Apply topic modeling to all texts with progress bar
topic_labels_list = [apply_and_print_progress(text) for text in tqdm(texts_list)]

# Convert results back to cuDF Series and add to main DataFrame
train_gf['topic_label'] = cudf.Series(topic_labels_list)

# Display total processed records
print(f"Total records processed: {count}")

In [ ]:
# Export the complete dataset with topic labels to CSV
train_gf.to_csv("/kaggle/working/aggregated_conversations_with_topicmodeling.csv", index=False)

In [ ]:
# Simplified approach: Apply topic modeling using pandas apply function
# Convert cuDF text column to pandas for processing
texts_pd_series = train_gf['text'].to_pandas()

def get_topic_label(text):
    """Extract topic label for a given text using the trained model"""
    topic, prob = topic_model_inference.transform(text)
    return topic_model_inference.topic_labels_[topic[0]]

# Apply topic modeling function to all texts using pandas apply
topic_labels = texts_pd_series.apply(get_topic_label)

# Convert results back to cuDF Series and add to main DataFrame
train_gf['topic_label'] = cudf.Series(topic_labels.values)

In [ ]:
# Final export of the complete dataset with topic modeling results
train_gf.to_csv("/kaggle/working/aggregated_conversations_with_topicmodeling.csv", index=False)